In [1]:
import psycopg2
import sys, os
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
conn_uri = 'postgresql://{user}:{user_pass}@{host}/{dataname2}'

In [3]:
t_visits_by_month = pd.read_sql_table("t_visits_by_month", conn_uri)

In [4]:
def sign_idx(idx_array):
    np_arr = np.array(idx_array)
    where = [np.where(np_arr == i) for i in [0,1]]
    [max_0_idx, max_1_idx] = [ np.max( w, initial = -1) for w in where]
    [min_0_idx, min_1_idx] = [ np.min( w, initial = np_arr.size)  for w in where]
    if (max_0_idx < min_1_idx and max_0_idx != -1) and min_1_idx != np_arr.size:
        return (max_0_idx == min_1_idx - 1, max_0_idx, 1)
    elif (max_1_idx < min_0_idx and max_1_idx != -1) and min_0_idx != np_arr.size:
        return (max_1_idx == min_0_idx - 1, max_1_idx, 0)
    else:
        all_same_idx = 1 - np.max(np.where(np.array([max_0_idx, max_1_idx])==-1),initial=-1)
        return (False, all_same_idx, 2)

In [5]:
def create_groups(visits_df):
    result = visits_df.copy()
    result[['0','1','2']] = result.apply(lambda x: sign_idx(x['months_idx']), axis=1, result_type='expand')
    result.drop(columns=['months_idx'], inplace=True)
    result.columns = ['is_special','kind','group']
    return result

In [11]:
%%capture
result = create_groups(t_visits_by_month)

In [207]:
# df = pd.DataFrame([{'months_idx':[1,1,0,0,0,0]},
#                    {'months_idx':[0,0,0,1,1]},
#                    {'months_idx':[0,0,0,0,0]},
#                    {'months_idx':[1,1,1,1,1]},
#                    {'months_idx':[0,0,0,0,0]},
#                    {'months_idx':[1,0,1,0,1]}]) 

In [14]:
result.size

12374580

In [152]:
sign_idx([1,1,0,0,0,0])

(True, 1, 0)

In [153]:
sign_idx([0,0,0,1,1])

(True, 2, 1)

In [154]:
sign_idx([0,0,0,0,0])

(False, 0, 2)

In [155]:
sign_idx([1,1,1,1,1])

(False, 1, 2)

In [156]:
sign_idx([0,0,0,0,0])

(False, 0, 2)

In [157]:
sign_idx([1,0,1,0,1])

(False, 2, 2)

In [15]:
result.head()

,is_special,kind,group
sname_place_id,,,
sg:000000c8f55d451ca1ae98bd057785a7,False,1,2
sg:000008c003ab41d9b6a8d1e58e4992ea,False,1,2
sg:00000f3789674e609a147b58323674d3,False,2,2
sg:00000fa74bd34d2086739db446b632e4,False,1,2
sg:00001b8625b64052888b8c2f2e3736bb,False,1,2


In [16]:
from sqlalchemy import create_engine
engine = create_engine(conn_uri)
result.to_sql(
            "entry_info",
               engine,
               if_exists='replace',
               schema='public',
               index=True,
               chunksize=50000
            )